In [1]:
import pandas as pd
import logging

# Log fayl manzili
log_path = r"C:\Users\Rasulbek907\Desktop\Project_8\Log\error_analysis.log"

# Log sozlamalari
logging.basicConfig(
    filename=log_path,
    filemode='a',  # Append mode
    format='%(asctime)s - %(levelname)s - %(message)s',
    level=logging.INFO
)

try:
    logging.info("CSV fayl o'qilmoqda:...")
    df = pd.read_csv(r"C:\Users\Rasulbek907\Desktop\Project_8\Data\Feature_Selection\Filtered_Features.csv")
    logging.info(f"Fayl muvaffaqiyatli o'qildi. Satırlar soni: {len(df)} ustunlar soni: {len(df.columns)}")
except Exception as e:
    logging.error(f"CSV faylni o'qishda xatolik: {e}")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322278 entries, 0 to 322277
Data columns (total 16 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Unnamed: 0            322278 non-null  int64  
 1   index                 322278 non-null  float64
 2   dateCrawled           322278 non-null  float64
 3   name                  322278 non-null  float64
 4   vehicleType           322278 non-null  float64
 5   yearOfRegistration    322278 non-null  float64
 6   model                 322278 non-null  float64
 7   monthOfRegistration   322278 non-null  float64
 8   fuelType              322278 non-null  float64
 9   brand                 322278 non-null  float64
 10  postalCode            322278 non-null  float64
 11  lastSeen              322278 non-null  float64
 12  abtest_control        322278 non-null  float64
 13  gearbox_automatik     322278 non-null  float64
 14  notRepairedDamage_ja  322278 non-null  float64
 15  

In [ ]:
import shap
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# --- Ma'lumotni tayyorlash ---
X = df.drop(columns=['Unnamed: 0', 'kilometer_cont'])
y = df['kilometer_cont']

# --- Train/Test split ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# --- Testdan faqat 2000 ta sample olish ---
X_test_small = X_test.sample(2000, random_state=42)

# --- Model ---
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# --- SHAP ---
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test_small)

# --- Summary ---
shap.summary_plot(shap_values, X_test_small, plot_type="bar")

# --- Waterfall 1 ta sample ---
# 0 indeksli sample uchun waterfall grafikini chizish
shap.plots.waterfall(explainer.shap_values(X_test_small)[0])